In [1]:
library(dplyr)
mydb <- src_sqlite("database.sqlite",create = F)
mydb


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



src:  sqlite 3.11.1 [database.sqlite]
tbls: Country, League, Match, Player, Player_Attributes, Team, Team_Attributes,
  sqlite_sequence

In [8]:
League <- tbl(mydb,"League")
head(League)

Source:   query [?? x 3]
Database: sqlite 3.11.1 [database.sqlite]

     id country_id                   name
  <int>      <int>                  <chr>
1     1          1 Belgium Jupiler League
2  1729       1729 England Premier League
3  4769       4769         France Ligue 1
4  7809       7809  Germany 1. Bundesliga
5 10257      10257          Italy Serie A
6 13274      13274 Netherlands Eredivisie

In [9]:
Match <- tbl(mydb,"Match")
head(Match)

Source:   query [?? x 115]
Database: sqlite 3.11.1 [database.sqlite]

     id country_id league_id    season stage                date match_api_id
  <int>      <int>     <int>     <chr> <int>               <chr>        <int>
1     1          1         1 2008/2009     1 2008-08-17 00:00:00       492473
2     2          1         1 2008/2009     1 2008-08-16 00:00:00       492474
3     3          1         1 2008/2009     1 2008-08-16 00:00:00       492475
4     4          1         1 2008/2009     1 2008-08-17 00:00:00       492476
5     5          1         1 2008/2009     1 2008-08-16 00:00:00       492477
6     6          1         1 2008/2009     1 2008-09-24 00:00:00       492478
# ... with 108 more variables: home_team_api_id <int>, away_team_api_id <int>,
#   home_team_goal <int>, away_team_goal <int>, home_player_X1 <int>,
#   home_player_X2 <int>, home_player_X3 <int>, home_player_X4 <int>,
#   home_player_X5 <int>, home_player_X6 <int>, home_player_X7 <int>,
#   home_player_X

In [10]:
library(tidyr)
test <- data.frame(Match %>%
  select(home_player_1 : away_player_11,league_id) )%>%
  gather(player, player_api_id,-league_id, na.rm = TRUE)

result <-test %>% 
group_by(player_api_id)%>% 
summarise(appearences=n(),LeagueNum=n_distinct(league_id))

arrange(result, desc(appearences))

player_api_id,appearences,LeagueNum
31293,300,1
41097,294,1
33764,286,1
42422,286,1
26295,282,2
31465,282,1
31432,275,1
26168,268,1
26399,262,1
94550,260,2


In [11]:
home <- data.frame(Match %>%
  select(home_team_api_id,home_team_goal) ) %>%
  group_by(team_api_id=home_team_api_id) %>%
  summarise(goals=sum(home_team_goal))

away <- data.frame(Match %>%
  select(away_team_api_id,away_team_goal) ) %>%
  group_by(team_api_id=away_team_api_id) %>%
  summarise(goals=sum(away_team_goal))

team_goals <- inner_join(x = home,y = away,by = "team_api_id")
total <- team_goals %>%
 group_by(team_api_id) %>%
 summarise(goals=goals.x+goals.y)

total

team_api_id,goals
1601,280
1773,126
1957,287
2033,138
2182,372
2183,184
2186,235
4049,35
4064,27
4087,180


In [12]:
Team <- tbl(mydb,"Team")
Teamname <-Team %>%
select(team_api_id, team_long_name)
Teamname

Source:   query [?? x 2]
Database: sqlite 3.11.1 [database.sqlite]

   team_api_id    team_long_name
         <int>             <chr>
1         9987          KRC Genk
2         9993      Beerschot AC
3        10000  SV Zulte-Waregem
4         9994  Sporting Lokeren
5         9984 KSV Cercle Brugge
6         8635    RSC Anderlecht
7         9991          KAA Gent
8         9998         RAEC Mons
9         7947     FCV Dender EH
10        9985 Standard de Liège
# ... with more rows

In [13]:
Team_Attributes <- tbl(mydb,"Team_Attributes")
Tattributes <-Team_Attributes %>%
mutate(Play=buildUpPlaySpeed + buildUpPlayDribbling+ buildUpPlayPassing,Creation=chanceCreationPassing+chanceCreationCrossing+chanceCreationShooting, Defense=defencePressure+defenceAggression+defenceTeamWidth) %>%
select(team_api_id, Play, Creation,Defense)
Tattributes



Source:   query [?? x 4]
Database: sqlite 3.11.1 [database.sqlite]

   team_api_id  Play Creation Defense
         <int> <int>    <int>   <int>
1         9930    NA      180     150
2         9930   156      181     145
3         9930   142      181     145
4         8485    NA      210     200
5         8485    NA      153     146
6         8485    NA      170     140
7         8485    NA      145     144
8         8485   184      181     143
9         8485   176      186     157
10        8576    NA      135     130
# ... with more rows

In [20]:
new_team <-inner_join(x = Teamname,y = Tattributes,by = "team_api_id")
final_team <-inner_join(x = data.frame(new_team),y =total,by = "team_api_id")
#final_team <-aggregate(cbind(Play,Creation,Defense,goals) ~ team_api_id + team_long_name, final_team, max)
write.csv(final_team,"team.csv")

In [34]:
Player <- tbl(mydb,"Player")
name <-Player %>%
select(player_api_id, player_name)
name

Source:   query [?? x 2]
Database: sqlite 3.11.1 [database.sqlite]

   player_api_id        player_name
           <int>              <chr>
1         505942 Aaron Appindangoye
2         155782    Aaron Cresswell
3         162549        Aaron Doran
4          30572      Aaron Galindo
5          23780       Aaron Hughes
6          27316         Aaron Hunt
7         564793         Aaron Kuhl
8          30895       Aaron Lennon
9         528212       Aaron Lennox
10        101042      Aaron Meijers
# ... with more rows

In [35]:
Player_Attributes <-tbl(mydb,"Player_Attributes")
attributes <-Player_Attributes %>%
mutate(GKoverall=gk_diving + gk_handling+ gk_kicking+ gk_positioning + gk_reflexes) %>%
select(player_api_id, overall_rating, GKoverall)
attributes

Source:   query [?? x 3]
Database: sqlite 3.11.1 [database.sqlite]

   player_api_id overall_rating GKoverall
           <int>          <int>     <int>
1         505942             67        43
2         505942             67        43
3         505942             62        43
4         505942             61        38
5         505942             61        38
6         155782             74        51
7         155782             74        51
8         155782             73        51
9         155782             73        51
10        155782             73        51
# ... with more rows

In [36]:
new_player <-inner_join(x = name,y = attributes,by = "player_api_id")
head(new_player,10)

Source:   query [?? x 4]
Database: sqlite 3.11.1 [database.sqlite]

   player_api_id        player_name overall_rating GKoverall
           <int>              <chr>          <int>     <int>
1         505942 Aaron Appindangoye             67        43
2         505942 Aaron Appindangoye             67        43
3         505942 Aaron Appindangoye             62        43
4         505942 Aaron Appindangoye             61        38
5         505942 Aaron Appindangoye             61        38
6         155782    Aaron Cresswell             74        51
7         155782    Aaron Cresswell             74        51
8         155782    Aaron Cresswell             73        51
9         155782    Aaron Cresswell             73        51
10        155782    Aaron Cresswell             73        51
# ... with more rows

In [37]:
final_player <- inner_join(x = data.frame(new_player),y = result,by = "player_api_id")
#final_player <-aggregate(cbind(overall_rating,GKoverall,appearences,LeagueNum) ~ player_api_id + player_name, final_player, max)
write.csv(final_player,"player.csv")

# Which are the top 10 players by overall rating?

In [38]:
player_overall <-final_player
player_overall <-aggregate(overall_rating ~ player_api_id + player_name, player_overall, max)
head(arrange(player_overall, desc(overall_rating)),10)

player_api_id,player_name,overall_rating
30981,Lionel Messi,94
30893,Cristiano Ronaldo,93
30717,Gianluigi Buffon,93
30829,Wayne Rooney,93
39989,Gregory Coupet,92
39854,Xavi Hernandez,92
30723,Alessandro Nesta,91
30955,Andres Iniesta,91
34520,Fabio Cannavaro,91
30657,Iker Casillas,91


# Which are the top 10 goalkeepers by sum of gk attributes?

In [39]:
player_gk <-final_player
player_gk <-aggregate(GKoverall ~ player_api_id + player_name, player_gk, max)
head(arrange(player_gk, desc(GKoverall)),10)

player_api_id,player_name,GKoverall
30717,Gianluigi Buffon,449
39989,Gregory Coupet,447
30859,Petr Cech,445
30657,Iker Casillas,442
27299,Manuel Neuer,440
30989,Julio Cesar,438
24503,Sebastian Frey,437
30726,Edwin van der Sar,436
182917,David De Gea,429
30660,Pepe Reina,428


# Which are the top 10 players by number of appearances

In [40]:
player_ap <-final_player
player_ap <-aggregate(appearences~ player_api_id + player_name, player_ap, max)
head(arrange(player_ap, desc(appearences)),10)

player_api_id,player_name,appearences
31293,Steve Mandanda,300
41097,Stephane Ruffier,294
33764,Gorka Iraizoz Moreno,286
42422,Samir Handanovic,286
26295,Hugo Lloris,282
31465,Tim Howard,282
31432,Joe Hart,275
26168,Cedric Carrasso,268
26399,Daniel Congre,262
94550,Moussa Sissoko,260


# Which are the top 10 players by number of leagues they played in?

In [41]:
player_l <-final_player
player_l <-aggregate(LeagueNum~ player_api_id + player_name, player_l, max)
head(arrange(player_l, desc(LeagueNum)),10)

player_api_id,player_name,LeagueNum
42119,Gelson Fernandes,6
178538,Filip Djuricic,5
116750,Ricky van Wolfswinkel,5
156534,Aleksandar Tonev,4
129918,Alfred Finnbogason,4
25816,Almen Abdi,4
93484,Aly Cissokho,4
46509,Angel Di Maria,4
96643,Bojan Krkic,4
38229,Bryan Ruiz,4


# Which are the top 10 teams by sum of build up play attributes?

In [22]:
team_play <-final_team
team_play <-aggregate(Play ~ team_api_id + team_long_name, team_play, max)
head(arrange(team_play, desc(Play)),10)

team_api_id,team_long_name,Play
9804,Torino,209
8600,Udinese,205
8194,Fortuna Düsseldorf,198
8033,Podbeskidzie Bielsko-Biała,193
8245,Korona Kielce,192
208931,Carpi,190
8024,Widzew Łódź,190
8244,Widzew Łódź,190
9830,FC Nantes,189
8543,Lazio,188


# Which are the top 10 teams by sum of chance creation attributes? 

In [26]:
team_creation <-final_team
team_creation <-aggregate(Creation ~ team_api_id + team_long_name, team_creation, max)
head(arrange(team_creation, desc(Creation)),10)

team_api_id,team_long_name,Creation
8543,Lazio,220
8697,SV Werder Bremen,216
9875,Napoli,215
8485,Aberdeen,210
8178,Bayer 04 Leverkusen,210
8658,Birmingham City,210
8191,Burnley,210
8344,Cardiff City,210
9860,Heart of Midlothian,210
8456,Manchester City,210


# Which are the top 10 teams by sum of defense attributes?

In [30]:
team_defense <-final_team
team_defense <-aggregate(Defense ~ team_api_id + team_long_name, team_defense, max)
head(arrange(team_defense, desc(Defense)),10)

team_api_id,team_long_name,Defense
8350,1. FC Kaiserslautern,210
8481,AS Nancy-Lorraine,210
9993,Beerschot AC,210
8658,Birmingham City,210
8483,Blackpool,210
8678,Bournemouth,210
8344,Cardiff City,210
9904,Hannover 96,210
8667,Hull City,210
1957,Jagiellonia Białystok,210


# Which are the top 10 teams by number of scored goals?

In [31]:
team_goal <-final_team
team_goal <-aggregate(goals ~ team_api_id + team_long_name, team_goal, max)
head(arrange(team_goal, desc(goals)),10)

team_api_id,team_long_name,goals
8634,FC Barcelona,849
8633,Real Madrid CF,843
9925,Celtic,695
9823,FC Bayern Munich,653
8640,PSV,652
8593,Ajax,647
9931,FC Basel,619
8456,Manchester City,606
8455,Chelsea,583
10260,Manchester United,582
